In [2]:
import scanpy as sc
import squidpy as sq
import pandas as pd

#sc.settings.verbosity = 3adata = sq.datasets.visium_hne_adata()
sc.settings.set_figure_params(dpi=80, facecolor="white")

/Users/romansankowski/mambaforge/lib/python3.10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/Users/romansankowski/mambaforge/lib/python3.10/site-packages/spatialdata/__init__.py:9: UserWarning: Geopandas was set to use PyGEOS, changing to shapely 2.0 with:

	geopandas.options.use_pygeos = True

If you intended to use PyGEOS, set the option to False.
  _check_geopandas_using_shapely()


In [8]:
counts =  pd.read_csv('data/Out_CtrlCtx4_Scale50_High_Prior_s50_sd50_conf08/segmentation_counts_filt_prior08.csv',
                     index_col=0) # feature matrix
coordinates = pd.read_csv('data/Out_CtrlCtx4_Scale50_High_Prior_s50_sd50_conf08/b1hi_ctrl_coord_prior08.csv',
                    index_col=0)  # spatial coordinates
#image = NULL  # image
labels = pd.read_csv('data/predictions_ctx4.csv',
                    index_col=0)

In [10]:
counts2 = counts.transpose()

### Create scanpy object 

In [11]:
adata = sc.AnnData(counts2.loc[labels.index], obsm={"spatial": coordinates.loc[labels.index],
                             "celltype" : labels})

### Standard clustering of scanpy object

In [12]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata)
adata

/Users/romansankowski/mambaforge/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/romansankowski/mambaforge/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/romansankowski/mambaforge/lib/python3.10/site-packages/umap/distanc

AnnData object with n_obs × n_vars = 109410 × 187
    obs: 'leiden'
    uns: 'log1p', 'pca', 'neighbors', 'umap', 'leiden'
    obsm: 'spatial', 'celltype', 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

### Compute spatial connectivity matrix

In [13]:
sq.gr.spatial_neighbors(adata)

In [14]:
## add celltype as clustering variable
adata.obs["celltype"] = list(labels['label'])
adata.obs["celltype"] = adata.obs["celltype"].astype('category')                           

KeyError: 'label'

### Run spatial enrichment analysis

In [ ]:
sq.gr.nhood_enrichment(adata, cluster_key="celltype")

In [ ]:
adata.uns["celltype_nhood_enrichment"]


In [ ]:
sq.pl.nhood_enrichment(adata, 
                       cluster_key="celltype", 
                       method="average", 
                       figsize=(9, 9),
                        save='cartana_celltype_neighborhood_enrichment.pdf')

### Add celltype interaction

In [ ]:
sq.gr.interaction_matrix(adata, cluster_key="celltype")

In [ ]:
adata.uns["celltype_interactions"]

In [ ]:
sq.pl.interaction_matrix(adata, cluster_key="celltype", method="average", figsize=(5, 5))

### Subset for parenchyma cells

In [ ]:
adata_ctx = adata[adata.obs['celltype'].isin(['Oligo', 'CAMs', 'Neuron','Astro','MG','Mural cells','Endothelial cells'])]
## run interaction analysis
sq.gr.interaction_matrix(adata_ctx, cluster_key="celltype")

In [ ]:
sq.pl.interaction_matrix(adata_ctx, 
                         cluster_key="celltype", 
                         method="average", 
                         figsize=(5, 5),
                        save='ctx_celltype_interactions.svg')

### Subset for LM cells 

In [ ]:
adata_lm = adata[adata.obs['celltype'].isin(['CAMs', 'Lymphocytes','Fibroblast','Mural cells','Endothelial cells'])]
## run interaction analysis
sq.gr.interaction_matrix(adata_lm, cluster_key="celltype")

In [ ]:
sq.pl.interaction_matrix(adata_lm, 
                         cluster_key="celltype", 
                         method="average", 
                         figsize=(9, 9),
                        save='lm_celltype_interactions.pdf')


### Export package versions

In [7]:
import sys
sys.modules.keys()
import types
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

In [8]:
!pip freeze > requirements.txt
